In [1]:
import open3d as o3d
import numpy as np
import copy

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## load cad model

In [5]:
mesh_target = o3d.io.read_triangle_mesh('../dataset/model/art_melon_without_handle.STL')
mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.4, origin=[0.0,0.0,0.0])
o3d.visualization.draw_geometries([mesh_target, mesh_frame])

In [6]:
# estimate normal and convert to pcd
mesh_target.compute_triangle_normals()
#mesh_target.orient_triangles()
pcd_target = mesh_target.sample_points_uniformly(number_of_points=10000)

In [7]:
o3d.visualization.draw_geometries([pcd_target])

## load point cloud

In [5]:
pcd_source = o3d.io.read_point_cloud('../dataset/d405/d405-2.ply')
mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1.0, origin=[0.0,0.0,0.0])
o3d.visualization.draw_geometries([pcd_source, mesh_frame])

In [6]:
# estimate normal
pcd_source.orient_normals_consistent_tangent_plane(k=15)
o3d.visualization.draw_geometries([pcd_source, mesh_frame])

## downsampling

In [7]:
voxel_size = 0.005
pcd_source_downsample = pcd_source.voxel_down_sample(voxel_size * 1.5)
pcd_target_downsample = pcd_target.voxel_down_sample(voxel_size * 1.5)

In [8]:
pcd_source_downsample.estimate_normals()
pcd_target_downsample.estimate_normals()

In [9]:
o3d.visualization.draw_geometries([pcd_source_downsample])
o3d.visualization.draw_geometries([pcd_target_downsample])

## crop bottle only from point cloud

In [10]:
# crop only bottle
min_bound = np.asarray([-0.5, -1.0, -1.0]) # [x_min, y_min, z_min]
max_bound = np.asarray([0.25, 1.0, 1.0]) # [x_max, y_max, z_max]
aabb = o3d.geometry.AxisAlignedBoundingBox(min_bound, max_bound)
pcd_source_cropped = pcd_source_downsample.crop(aabb)
o3d.visualization.draw_geometries([pcd_source_cropped, mesh_frame])

In [11]:
# remove table
plane_model, plane_inlier_indeces = pcd_source_cropped.segment_plane(distance_threshold=0.01, ransac_n=3, num_iterations=500)
pcd_plane = pcd_source_cropped.select_by_index(plane_inlier_indeces)
pcd_plane.paint_uniform_color([1, 0, 0])
o3d.visualization.draw_geometries([pcd_source_cropped, pcd_plane])
pcd_source_removed_plane = pcd_source_cropped.select_by_index(plane_inlier_indeces, invert=True)
o3d.visualization.draw_geometries([pcd_source_removed_plane])

In [12]:
# remove outlier
cl, inlier_indeces = pcd_source_removed_plane.remove_statistical_outlier(nb_neighbors=60, std_ratio=2.5)
pcd_source_removed_outlier = pcd_source_removed_plane.select_by_index(inlier_indeces)
o3d.visualization.draw_geometries([pcd_source_removed_outlier])

## rotate cad at -90 deg

In [13]:
rad = np.radians(-90)
R = o3d.geometry.get_rotation_matrix_from_axis_angle(np.array([rad, 0, 0]))
pcd_target_downsample.rotate(R)

PointCloud with 2336 points.

## align center

In [14]:
pcd_source_removed_outlier.translate(-pcd_source_removed_outlier.get_center())
pcd_target_downsample.translate(-pcd_target_downsample.get_center())

PointCloud with 2336 points.

In [15]:
mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.4, origin=[0.0,0.0,0.0])
o3d.visualization.draw_geometries([pcd_source_removed_outlier, pcd_target_downsample, mesh_frame])

## fpfh feature

In [16]:
fpfh_source = o3d.pipelines.registration.compute_fpfh_feature(
    pcd_source_removed_outlier, o3d.geometry.KDTreeSearchParamHybrid(radius=5.0*voxel_size, max_nn=30))
fpfh_target = o3d.pipelines.registration.compute_fpfh_feature(
    pcd_target_downsample, o3d.geometry.KDTreeSearchParamHybrid(radius=5.0*voxel_size, max_nn=30))

## ransac

In [17]:
distance_threshold = voxel_size * 1.5
result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
    pcd_source_removed_outlier, pcd_target_downsample, fpfh_source, fpfh_target, True, distance_threshold,
    o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
    ransac_n=3,
    checkers = [o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9),
                o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold),
                o3d.pipelines.registration.CorrespondenceCheckerBasedOnNormal(np.pi / 2)
               ],
    criteria = o3d.pipelines.registration.RANSACConvergenceCriteria(100000, 0.999)
)

[Open3D WARNING] Too few correspondences (33) after mutual filter, fall back to original correspondences.


In [18]:
result.transformation

array([[-0.95425169, -0.07689192, -0.28894869,  0.0054677 ],
       [ 0.22037008, -0.83400782, -0.50583395,  0.20098501],
       [-0.20209092, -0.54636855,  0.81279805,  0.03268796],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

## show result of RANSAC

In [19]:
# define draw func
def draw_registration_result(source, target, transformation):
    pcds = list()
    for s in source:
        temp = copy.deepcopy(s)
        pcds.append(temp.transform(transformation))
    pcds += target
    o3d.visualization.draw_geometries(pcds)

# rename results
pcd_source_result = pcd_source_removed_outlier.paint_uniform_color([1, 0, 0])
pcd_target_result = pcd_target_downsample.paint_uniform_color([0, 0, 1])

# show result
draw_registration_result([pcd_source_result], [pcd_target_result], result.transformation)

## icp

In [20]:
# icp
distance_threshold = 0.20 # 10cm
initial_trans = np.identity(4)

criteria = o3d.pipelines.registration.ICPConvergenceCriteria(
    max_iteration=500,  
    relative_fitness=1e-6, 
    relative_rmse=1e-6
)

reg_p2l = o3d.pipelines.registration.registration_icp(
    pcd_source_removed_outlier, pcd_target_downsample, distance_threshold, initial_trans,
    o3d.pipelines.registration.TransformationEstimationPointToPoint(),
    criteria=criteria
)

## show result of icp

In [21]:
# rename results
pcd_source_result = pcd_source_removed_outlier.paint_uniform_color([1, 0, 0])
pcd_target_result = pcd_target_downsample.paint_uniform_color([0, 0, 1])

# show result
draw_registration_result([pcd_source_result], [pcd_target_result], initial_trans)
draw_registration_result([pcd_source_result], [pcd_target_result], reg_p2l.transformation)

In [22]:
reg_p2l.transformation

array([[ 0.98848398, -0.15090946,  0.01121368,  0.00741515],
       [ 0.1500299 ,  0.98700243,  0.05759544,  0.01205326],
       [-0.01975963, -0.05524978,  0.99827702,  0.0423772 ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])